In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 128
#model_name = "VGG16_quant"
#model = VGG16_quant()
model_name = "resnet20_quant"
model = resnet20_quant()
print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    #adjust_list = [150, 225]
    adjust_list = [80, 120]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
ResNet_Cifar(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()

In [7]:
model.conv1 = QuantConv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
print(model.conv1)

for layer in model.modules():
    if isinstance(layer, torch.nn.Conv2d):
        layer.bit = 2
        layer.weight_quant = weight_quantize_fn(2)
        layer.act_alq = act_quantization(2)
        layer.act_alpha = torch.nn.Parameter(torch.tensor(4.0))

QuantConv2d(
  3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)


In [8]:
for layer in model.modules():
    if isinstance(layer, torch.nn.Conv2d):
        if layer.layer_type == "QuantConv2d":
            print(layer.bit)
            print(layer.weight_quant.w_bit)

2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1


In [9]:
print(model)

ResNet_Cifar(
  (conv1): QuantConv2d(
    3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
    (weight_quant): weight_quantize_fn()
  )
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (we

In [10]:
# This cell won't be given, but students will complete the training

lr = 4e-2
weight_decay = 1e-4
epochs = 100
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name) + "_2bit"
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: [0][0/391]	Time 0.817 (0.817)	Data 0.745 (0.745)	Loss 2.5817 (2.5817)	Prec 11.719% (11.719%)
Epoch: [0][100/391]	Time 0.043 (0.052)	Data 0.002 (0.009)	Loss 1.4871 (1.6891)	Prec 43.750% (38.428%)
Epoch: [0][200/391]	Time 0.048 (0.048)	Data 0.002 (0.006)	Loss 1.5641 (1.6262)	Prec 46.094% (40.909%)
Epoch: [0][300/391]	Time 0.043 (0.047)	Data 0.002 (0.004)	Loss 1.5567 (1.5960)	Prec 46.875% (42.112%)
Validation starts
Test: [0/79]	Time 0.460 (0.460)	Loss 1.4798 (1.4798)	Prec 50.000% (50.000%)
 * Prec 46.840% 
best acc: 46.840000
Epoch: [1][0/391]	Time 0.650 (0.650)	Data 0.587 (0.587)	Loss 1.4545 (1.4545)	Prec 42.969% (42.969%)
Epoch: [1][100/391]	Time 0.043 (0.052)	Data 0.002 (0.008)	Loss 1.5685 (1.4578)	Prec 45.312% (46.566%)
Epoch: [1][200/391]	Time 0.041 (0.049)	Data 0.002 (0.005)	Loss 1.4801 (1.4450)	Prec 50.781% (47.322%)
Epoch: [1][300/391]	Time 0.046 (0.047)	Data 0.002 (0.004)	Loss 1.4710 (1.4346)	Prec 46.875% (47.755%)
Validation starts
Test: [0/79]	Time 0.663 (0.663)	Loss 1.

In [ ]:
# HW

#  1. Train with 4 bits for both weight and activation to achieve >90% accuracy
#  2. Find x_int and w_int for the 2nd convolution layer
#  3. Check the recovered psum has similar value to the un-quantized original psum
#     (such as example 1 in W3S2)

In [11]:
#PATH = "result/VGG16_quant/model_best.pth.tar"
PATH = "result/resnet20_quant_2bit/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 7963/10000 (80%)



In [12]:
#send an input and grap the value by using prehook like HW3
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()

for layer in model.modules():
    if isinstance(layer, torch.nn.Conv2d):
        print("prehooked")
        layer.register_forward_pre_hook(save_output)       ## Input for the module will be grapped       
####################################################

dataiter = iter(trainloader)
images, labels = dataiter.next()
images = images.to(device)
out = model(images)
print("1st convolution's input size:", save_output.outputs[0][0].size())
print("2nd convolution's input size:", save_output.outputs[1][0].size())

prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
prehooked
1st convolution's input size: torch.Size([128, 3, 32, 32])
2nd convolution's input size: torch.Size([128, 16, 32, 32])


In [13]:
conv2= model.layer1[0].conv1
#print(conv2.weight_q)
#print(conv2.weight_quant.wgt_alpha.data.item())
#print(conv2.weight_quant.wgt_alpha.data.item()/ (2**4 - 1))
print(conv2.weight_quant.w_bit)

1


In [15]:
w_bit = 2
#weight_q = model.features[3].weight_q # quantized value is stored during the training

weight_q = conv2.weight_q
#print(conv2.weight)
w_alpha = conv2.weight_quant.wgt_alpha.data.item()
w_delta = w_alpha / (2**(w_bit - 1) - 1)
weight_int = weight_q / w_delta
print(weight_int) # you should see clean integer numbers

tensor([[[[-0., -0., -0.],
          [-0., -0., -0.],
          [-0., -0., -0.]],

         [[-0.,  0.,  0.],
          [-0.,  0., -0.],
          [ 1.,  1.,  1.]],

         [[-0., -0., -0.],
          [-0., -0., -0.],
          [-0., -0., -0.]],

         ...,

         [[-0., -0., -0.],
          [-0., -0., -0.],
          [-0., -0., -0.]],

         [[-0., -0., -0.],
          [-0., -0., -0.],
          [-0., -0., -0.]],

         [[-0., -0., -0.],
          [-0., -0., -0.],
          [-0., -0., -0.]]],


        [[[-0., -0., -0.],
          [-0., -0., -0.],
          [-0., -0., -0.]],

         [[-0., -0., -0.],
          [-0., -0., -0.],
          [-0., -0., -0.]],

         [[-0., -0., -0.],
          [-0., -0., -0.],
          [-0., -0., -0.]],

         ...,

         [[-0., -0., -0.],
          [-0., -0., -0.],
          [-0., -0., -0.]],

         [[-0., -0., -0.],
          [-0., -0., -0.],
          [-0., -0., -0.]],

         [[-0., -0., -0.],
          [-0., -0., -0.],
 

In [ ]:
#print(save_output.outputs[1][0])
#print(conv2.act_alpha.data.item())


In [16]:
x_bit = 2
x =  save_output.outputs[1][0] # input of the 2nd conv layer
x_alpha  = conv2.act_alpha.data.item()
x_delta = x_alpha / (2**x_bit - 1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q / x_delta
print(x_int) # you should see clean integer numbers 

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[2., 2., 2.,  ..., 2., 2., 0.],
          [2., 2., 2.,  ..., 2., 2., 0.],
          [2., 2., 2.,  ..., 2., 2., 0.],
          ...,
          [3., 3., 3.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [2., 2., 2.,  ..., 2., 2., 2.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         ...,

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 

In [17]:
conv_int = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)

output_int =  conv_int(x_int)
output_recovered = output_int * x_delta * w_delta
print(output_recovered)

tensor([[[[ 26.5919,  21.6060,  21.6060,  ...,  21.6060,  21.6060,  18.2820],
          [ 29.9159,  24.9299,  24.9299,  ...,  24.9299,  24.9299,  21.6060],
          [ 29.9159,  24.9299,  24.9299,  ...,  26.5919,  24.9299,  21.6060],
          ...,
          [ 41.5499,  38.2259,  36.5639,  ...,  24.9299,  16.6200,  14.9580],
          [ 14.9580,  11.6340,  11.6340,  ...,  13.2960,  14.9580,  14.9580],
          [ 19.9440,  19.9440,  19.9440,  ...,  26.5919,  24.9299,  24.9299]],

         [[  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
          [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
          [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
          ...,
          [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
          [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
          [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000]],

         [[-36.5639, -41.5499,

In [ ]:
#### input floating number / weight quantized version

#conv_ref = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
#conv_ref.weight = torch.nn.parameter.Parameter(weight_int)

#output_ref = conv_ref(x)
#print(output_ref)

In [18]:
#### input floating number / weight floating number version

conv_ref = torch.nn.Conv2d(in_channels = 16, out_channels=16, kernel_size = 3, bias = False)
#weight = model.features[3].weight
#mean = weight.data.mean()
#std = weight.data.std()
#conv_ref.weight = torch.nn.parameter.Parameter(weight.add(-mean).div(std))
conv_ref.weight = torch.nn.parameter.Parameter(weight_q)
output_ref = conv_ref(x)
print(output_ref)


tensor([[[[  31.3361,   25.7512,   26.3097,  ...,   26.8682,   28.2257,
             22.8780],
          [  41.2339,   35.6490,   33.1746,  ...,   36.2075,   35.6490,
             32.7758],
          [  41.2339,   35.6490,   35.6490,  ...,   36.7660,   35.6490,
             32.7758],
          ...,
          [  68.3694,   62.5228,   57.1892,  ...,   34.0343,   34.1165,
             37.0579],
          [  20.4976,   15.2951,   14.1781,  ...,   20.4259,   29.6345,
             34.5834],
          [  24.7818,   31.2832,   31.2832,  ...,   42.8110,   45.2855,
             45.2855]],

         [[   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000],
          [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000],
          [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000],
          ...,
          [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
              0.0000],
          [   0.00

In [19]:
difference = abs( output_ref - output_recovered )
print(difference.mean()) 

tensor(17.8815, device='cuda:0', grad_fn=<MeanBackward0>)
